In [ ]:
! pip install geopandas numpy rasterio pyproj shapely scipy xarray

In [ ]:
import requests
import json
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
import xarray as xr
from datetime import datetime
from geojson import Feature, FeatureCollection, Point
from rasterio.transform import from_origin
from shapely.geometry import Point
from pyproj import Transformer
from scipy.spatial import cKDTree

In [ ]:
url = "https://api.weather.gc.ca/collections/aqhi-forecasts-realtime/items?f=json&limit=100000"
response = requests.get(url)
data = response.json()

In [ ]:
print(json.dumps(data, indent=2))

In [ ]:
features = []

for item in data.get("features", []):
    props = item["properties"]
    geom = item["geometry"]

    feature = Feature(
        geometry=Point((geom["coordinates"][0], geom["coordinates"][1])),
        properties={
            "location": props.get("location_name_en", "Unknown"),
            "location_id": props.get("location_id", "N/A"),
            "aqhi": props.get("aqhi", -1),
            "forecast_time": props.get("forecast_datetime", "Unknown"),
            "forecast_issue": props.get("publication_datetime", "Unknown")
        }
    )
    features.append(feature)

geojson_data = FeatureCollection(features)

In [ ]:
gdf = gpd.GeoDataFrame.from_features(geojson_data.features)

gdf.head(50)

In [ ]:
gdf["forecast_time"] = pd.to_datetime(gdf["forecast_time"])
gdf["forecast_issue"] = pd.to_datetime(gdf["forecast_issue"])

gdf["hours_after_issue"] = (gdf["forecast_time"] - gdf["forecast_issue"]).dt.total_seconds() / 3600
gdf_filtered = gdf[gdf["hours_after_issue"] == 1].copy()
gdf_filtered.drop(columns=["hours_after_issue"], inplace=True)

gdf_filtered.head(10)

In [ ]:
gdf_filtered.to_file("data/aqhi_forecast.geojson", driver="GeoJSON")

print(f"Saved {len(gdf_filtered)} AQHI forecast points to file aqhi_forecast.geojson")

In [ ]:
def interpolate_aqhi_from_gdf(
    gdf: gpd.GeoDataFrame, 
    grid_res: int = 2000, 
    k: int = 10, 
    power: int = 2
) -> xr.DataArray:
    
    gdf = gdf[gdf.geometry.notnull()].copy()
    gdf = gdf[gdf["aqhi"].notnull()].copy()
    gdf.set_crs(epsg=4326, inplace=True, allow_override=True)
    gdf = gdf.to_crs(epsg=3978)

    coords = np.array([[geom.x, geom.y] for geom in gdf.geometry])
    values = gdf["aqhi"].astype(float).values

    valid_mask = np.isfinite(coords).all(axis=1) & np.isfinite(values)
    coords = coords[valid_mask]
    values = values[valid_mask]

    if len(coords) == 0:
        raise ValueError("Interpolation failed: no valid AQHI monitoring points found.")

    xmin, ymin, xmax, ymax = gdf.total_bounds
    if np.isnan([xmin, ymin, xmax, ymax]).any():
        raise ValueError("NaN detected in total_bounds.")

    x_coords = np.arange(xmin, xmax, grid_res)
    y_coords = np.arange(ymin, ymax, grid_res)
    xx, yy = np.meshgrid(x_coords, y_coords)
    grid_points = np.c_[xx.ravel(), yy.ravel()]

    def idw_interpolation(xy_known, values_known, xy_grid, k=10, power=2):
        tree = cKDTree(xy_known)
        dists, idxs = tree.query(xy_grid, k=k)
        weights = 1 / np.power(dists + 1e-10, power)
        return np.sum(weights * values_known[idxs], axis=1) / np.sum(weights, axis=1)

    interpolated_vals = idw_interpolation(coords, values, grid_points)
    grid_array = interpolated_vals.reshape((len(y_coords), len(x_coords)))

    da = xr.DataArray(
        grid_array,
        coords={"y": yy[:, 0], "x": xx[0, :]},
        dims=["y", "x"],
        name="aqhi"
    )

    return da

In [ ]:
aqhi_grid = interpolate_aqhi_from_gdf(gdf_filtered, grid_res=2000)
aqhi_grid.plot()

In [ ]:
aqhi_grid.to_netcdf("data/aqhi_grid.nc")